In [1]:
import gensim
import os

wd=os.getcwd()

#foldermain=wd+'/Experiments/'
foldermain='../Experiments/'
#folderpath='../EXPERIMENTS/MODELS/'
folderpath=foldermain+'/MODELS/'

dictionary = gensim.corpora.Dictionary.load(folderpath+'MultiCore.dict')
corpus=gensim.corpora.MmCorpus(folderpath+'MultiCoreCorpus.mm')
modelfile=''
# for f in os.listdir(folderpath):
#     if('.model' in f and '.model.state' not in f):
#         modelfile=f
for f in os.listdir(folderpath):
    if ('.model' in f):
        filetype = f.split('.')[-1]
        if filetype not in ['state','id2word','npy']:
            modelfile = f
print(modelfile)       
model_test= gensim.models.ldamodel.LdaModel.load(folderpath+modelfile)

4_43.model


In [2]:
#show word probabilities for each topic
X=model_test.show_topics(num_topics=50, num_words=5, log=False, formatted=False)
test=[(x[0],[y[0] for y in x[1]]) for x in X]
topicDesc={}

for t in test:
    topicstr=' + '.join(str(e) for e in t[1])
    #print topicstr
    topicDesc[t[0]]=topicstr

model_test.show_topics(num_topics=50, num_words=5, log=False, formatted=False)

[(0,
  [(u'minikanren', 0.035452675),
   (u'oper', 0.035452664),
   (u'languag', 0.023657305),
   (u'program', 0.02365667),
   (u'api', 0.02365567)]),
 (1,
  [(u'microlightj', 0.07491697),
   (u'languag', 0.07491596),
   (u'librari', 0.037556354),
   (u'refer', 0.037556086),
   (u'usag', 0.03755554)]),
 (2,
  [(u'plugin', 0.089471005),
   (u'uri', 0.044793587),
   (u'horizontalnav', 0.044793587),
   (u'jqueri', 0.044793587),
   (u'fit', 0.02245484)]),
 (3,
  [(u'presentzj', 0.080967374),
   (u'github', 0.040588852),
   (u'websit', 0.040588807),
   (u'heart', 0.04058868),
   (u'softwar', 0.04058866)])]

In [3]:
#WRITETOFILE
folder=foldermain+'REPOPROPS/'

for k in topicDesc.keys():
    line=str(k)+','+topicDesc[k]
    fout=open(folder+'topickeys.csv','a')
    fout.write(line+'\n')
    fout.close()
    


In [4]:
#LOAD DOCUMENTS
import os
foldername=foldermain+'/FILTEREDRM/PREPROCESSED/'
fileList = os.listdir(foldername)
docdict={}
for fn in fileList:
    f = open(foldername + fn, 'r')
    txt = str(f.read())
    docdict[fn]=txt.split()
    f.close()

In [5]:
#FIND TOPIC
import numpy as np
docTopicDict={}
for d in docdict.keys():
    docProbs=model_test[[dictionary.doc2bow(docdict[d])]]
    currrdocProb = [0]*49
    for p in docProbs[0]:
        currrdocProb[p[0]]=p[1]
        doc_topic = np.array(currrdocProb)
        topic=np.array(doc_topic).argmax()
        docTopicDict[d.strip().replace('.txt','')]=topic

In [6]:
#WRITE DOCTOPIC TO FILES
folder=foldermain+'REPOPROPS/'
for k in docTopicDict:
    line=k+','+str(docTopicDict[k])
    fout=open(folder+'doctopic.csv','a')
    fout.write(line+'\n')
    fout.close()

In [7]:
#LOAD DOC/proj/repo NAMEs
#repdata=os.listdir(foldermain+'/HTMLSORG')
repdata=os.listdir(foldermain+'/FILTEREDRM/PREPROCESSED/')


In [8]:
#COMBINE DATA
fnames=docTopicDict.keys()
cnt=0
folder=foldermain+'REPOPROPS/'
comb=open(folder+'comb2desc.csv','a')
for repoline in repdata:
    #fkey=repoline.split(',')[0]
    fkey=repoline.replace('.txt','')   
    if(fkey in fnames):
        currtopic=docTopicDict[fkey]
        newline=repoline.replace('\n','')+','+str(currtopic)+','+str(topicDesc[currtopic])
    else:
        newline=repoline.replace('\n','')+',TOPICASSIGNED,TOPICWORDS'
    comb.write(newline+'\n')
comb.close()